In [ ]:
# default_exp problem_types.pretrain
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
# test setup
# import tensorflow as tf
# import numpy as np
# from m3tl.test_base import TestBase
# from m3tl.input_fn import train_eval_input_fn
# from m3tl.test_base import test_top_layer
# test_base = TestBase()
# params = test_base.params

# params.assign_problem('weibo_fake_ner&weibo_fake_cls|weibo_fake_multi_cls|weibo_masklm|weibo_pretrain',base_dir=test_base.tmpckptdir)

# hidden_dim = params.bert_config.hidden_size

# train_dataset = train_eval_input_fn(params=params)
# one_batch = next(train_dataset.as_numpy_iterator())


# Pretrain(pretrain)

This module includes neccessary part to register pretrain problem type.

## Imports and utils


In [ ]:
# export
from typing import Dict, List, Tuple

from loguru import logger
import tensorflow as tf
import transformers
from m3tl.base_params import BaseParams
from m3tl.problem_types.utils import empty_tensor_handling_loss
from m3tl.special_tokens import PREDICT
from m3tl.utils import (LabelEncoder, gather_indexes, get_phase,
                        variable_summaries)
from transformers import TFSharedEmbeddings


## Top Layer

In [ ]:
# export

class PreTrain(tf.keras.Model):
    def __init__(self, params: BaseParams, problem_name: str, input_embeddings: tf.Tensor=None, share_embedding=True):
        super(PreTrain, self).__init__(name=problem_name)
        self.problem_name = problem_name
        self.params = params
        self.nsp = transformers.models.bert.modeling_tf_bert.TFBertNSPHead(
            self.params.bert_config)

        if share_embedding is False:
            self.vocab_size = self.params.bert_config.vocab_size
            self.share_embedding = False
        else:
            word_embedding_weight = input_embeddings.word_embeddings
            self.vocab_size = word_embedding_weight.shape[0]
            embedding_size = word_embedding_weight.shape[-1]
            share_valid = (self.params.bert_config.hidden_size ==
                        embedding_size)
            if not share_valid and self.params.share_embedding:
                logger.warning(
                    'Share embedding is enabled but hidden_size != embedding_size')
            self.share_embedding = self.params.share_embedding & share_valid

        if self.share_embedding:
            self.share_embedding_layer = TFSharedEmbeddings(
                vocab_size=word_embedding_weight.shape[0], hidden_size=word_embedding_weight.shape[1])
            self.share_embedding_layer.build([1])
            self.share_embedding_layer.weight = word_embedding_weight
        else:
            self.share_embedding_layer = tf.keras.layers.Dense(self.vocab_size)

    def call(self,
             inputs: Tuple[Dict[str, Dict[str, tf.Tensor]], Dict[str, Dict[str, tf.Tensor]]]) -> Tuple[tf.Tensor, tf.Tensor]:
        mode = get_phase()
        features, hidden_features = inputs

        # compute logits
        nsp_logits = self.nsp(hidden_features['pooled'])

        # masking is done inside the model
        seq_hidden_feature = hidden_features['seq']
        if mode != PREDICT:
            positions = features['masked_lm_positions']

            # gather_indexes will flatten the seq hidden_states, we need to reshape
            # back to 3d tensor
            input_tensor = gather_indexes(seq_hidden_feature, positions)
            shape_tensor = tf.shape(positions)
            shape_list = tf.concat(
                [shape_tensor, [seq_hidden_feature.shape.as_list()[-1]]], axis=0)
            input_tensor = tf.reshape(input_tensor, shape=shape_list)
            # set_shape to determin rank
            input_tensor.set_shape(
                [None, None, seq_hidden_feature.shape.as_list()[-1]])
        else:
            input_tensor = seq_hidden_feature
        if self.share_embedding:
            mlm_logits = self.share_embedding_layer(
                input_tensor, mode='linear')
        else:
            mlm_logits = self.share_embedding_layer(input_tensor)

        if self.params.detail_log:
            for weight_variable in self.weights:
                variable_summaries(weight_variable, self.problem_name)

        if mode != PREDICT:
            nsp_labels = features['next_sentence_label_ids']
            mlm_labels = features['masked_lm_ids']
            mlm_labels.set_shape([None, None])
            # compute loss
            nsp_loss = empty_tensor_handling_loss(
                nsp_labels, nsp_logits,
                tf.keras.losses.sparse_categorical_crossentropy)
            mlm_loss_layer = transformers.modeling_tf_utils.TFMaskedLanguageModelingLoss()
            # mlm_loss = tf.reduce_mean(
            #     mlm_loss_layer.compute_loss(mlm_labels, mlm_logits))

            # add a useless from_logits argument to match the function signature of keras losses.
            def loss_fn_wrapper(labels, logits, from_logits=True):
                return mlm_loss_layer.compute_loss(labels, logits)
            mlm_loss = empty_tensor_handling_loss(
                mlm_labels,
                mlm_logits,
                loss_fn_wrapper
            )
            loss = nsp_loss + mlm_loss
            self.add_loss(loss)

        return (tf.sigmoid(nsp_logits), tf.nn.softmax(mlm_logits))

In [ ]:
# test_top_layer(PreTrain, problem='weibo_fake_cls', params=params, sample_features=one_batch, hidden_dim=hidden_dim, share_embedding=False)

## Get or make label encoder function


In [ ]:
# export
def pretrain_get_or_make_label_encoder_fn(params: BaseParams, problem: str, mode: str, label_list: List[str], *args, **kwargs) -> LabelEncoder:
    params.set_problem_info(problem=problem, info_name='num_classes', info=1)
    return None

## Label handing function

In [ ]:
# export
def pretrain_label_handling_fn(target, label_encoder=None, tokenizer=None, decoding_length=None, *args, **kwargs):
    return None, None

